In [1]:
import re
from urllib import error
from html import unescape
import requests
import json 
from bs4 import BeautifulSoup
import time
import sqlite3

In [12]:
seen =[]

In [2]:
def download(method,url,param=None,data=None, timeout=1, maxretries=3):
    headers ={"Accept-Language": "ko-KR",
              "Client-Id": "kimne78kx3ncx6brgo4mv6wki5h1ko",
              "Content-Type": "text/plain;charset=UTF-8",
              "Origin": "https://www.twitch.tv",
              "Referer": "https://www.twitch.tv/search?term=%ED%92%8D%EC%9B%94%EB%9F%89%2F459527458",
              "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36",
              "X-Device-Id": "1896bda3b7a429f1"}
                #  개발자 도구에서 qql의 request헤더부분 설정함
    try:
        resp=requests.request(method, url,params=param,data=data, headers=headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500<=e.response.status_code<600 and maxretries>0:
            print(maxretries)
            
            download(method,url,param,data,timeout,maxretries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [121]:
# 개발자 도구에서 qql의 request payload설정부분
def set_param_for_find_bj_id(name_bj):
    return {"operationName":"SearchResultsPage_SearchResults","variables":{"query":name_bj,"options":None},"extensions":{"persistedQuery":{"version":1,"sha256Hash":"e5763309c0c0d14e7b59d5da0975794469cab4182e85b8f220820a43ff15905c"}}}
    

In [130]:
def get_bj_id(name_bj):
    data = set_param_for_find_bj_id(name_bj)
    #post를 보내기위해 payload를 불러온다.
    data = json.dumps(data)
    #json형식이 object안에 object가 있는 형태라면 dumps를 써야한다. 이유는 모른다.
    html = download("post","https://gql.twitch.tv/gql",data = data)
    dom = html.json()
    return dom['data']['searchFor']['channels']['items'][0]['login']

In [132]:
# 개발자 도구에서 qql의 request payload설정부분
def set_param_for_find_all_id(id_bj):
    return {"operationName":"FilterableVideoTower_Videos","variables":{"limit":30,"channelOwnerLogin":id_bj,"broadcastType":None,"videoSort":"TIME"},"extensions":{"persistedQuery":{"version":1,"sha256Hash":"2023a089fca2860c46dcdeb37b2ab2b60899b52cca1bfa4e720b260216ec2dc6"}}}
    

In [138]:
def get_id(data,lst=None):
    if lst ==None:
        lst=[]
    #data = search_twi(id_bj)
    data1 = json.dumps(data)
    html = download("post","https://gql.twitch.tv/gql",data = data1)    
    dom =html.json()
    dom['data']['user']['videos']['edges'][len(dom['data']['user']['videos']['edges'])-1]
    lst.extend([_['node']['id'] for _ in dom['data']['user']['videos']['edges']])
    if dom['data']['user']['videos']['pageInfo']['hasNextPage'] == True:
        data['variables']['cursor']=dom['data']['user']['videos']['edges'][len(dom['data']['user']['videos']['edges'])-1]['cursor']
        get_id(data,lst)
    return lst

In [139]:
def get_all_id(bj_name):
    bj_id = get_bj_id(bj_name)
    data = set_param_for_find_all_id(bj_id)
    return get_id(data)

In [141]:
res = get_all_id('엠비션')

In [143]:
len(set(res))

267